<a href="https://colab.research.google.com/github/AlyssonQueiroz/imersaoai/blob/main/clinica_de_prevencao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('chave')
genai.configure(api_key=api_key)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
#exemplo de embedding
title = "Alzheimer"
sample_text = ("Alzheimer")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.026024096, -0.06048553, -0.053410962, 0.0038181804, 0.05546338, 0.036810264, 0.049905196, -0.023732197, -0.012320285, -0.007918594, 0.015211069, 0.017042002, -0.031246824, 0.03799316, -0.00043711628, -0.037928067, -0.004044279, 0.0624441, -0.00051848445, -0.005155016, 0.037847824, 0.035235, -0.03447168, 0.01089532, -0.0049013803, -0.028202778, 0.006542588, -0.07384381, -0.03191944, 0.019425275, -0.041346792, -0.007260211, -0.031244757, 0.037970662, 0.0045384676, -0.049012784, 0.017450845, -0.004917359, -0.019279035, 0.027587704, -0.0022164881, -0.02537765, -0.073227406, 0.0021984384, 0.046975598, -0.016156198, -0.02186938, 0.067906216, 0.03867791, -0.07147629, 0.062640935, 0.0065648817, 0.058510657, -0.041550357, -0.017455326, -0.063029274, 0.0045421757, -0.016958944, -0.016972436, -0.00081036065, -0.033856235, 0.020362034, -0.028902069, 0.040578622, -0.02957268, -0.050106287, -0.039596017, 0.0075997226, 0.033505626, 0.009419102, 0.0074651726, 0.013096936, 0.05515317, 

In [ ]:
#listagem de documentos
DOCUMENT1 = {
    "Título": "Prevenção de Diabetes",
    "Conteúdo": "A obesidade é um fator de risco importante para a diabetes. Os exercícios ajudam a controlar os níveis de açúcar no sangue e melhoram a sensibilidade à insulina. Limitar alimentos processados, açúcares adicionados e gorduras saturadas. Optar por frutas, vegetais, "
}
DOCUMENT2 = {
    "Título": "Prevenção de Cardiopatias",
    "Conteúdo": "Fumar danifica os vasos sanguíneos e aumenta o risco de doenças cardíacas. O colesterol alto pode obstruir as artérias e aumentar o risco de doenças cardíacas."
}
DOCUMENT3 = {
    "Título": "Como prevenir Alzheimer",
    "Conteúdo": "Uma dieta mediterrânea, rica em frutas, verduras, peixes e azeite, pode ajudar a reduzir o risco de Alzheimer."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]

In [ ]:
model = "models/embedding-001"

In [ ]:
def embed_fn(title, text):
  return genai.embed_content(model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]


In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Prevenção de Diabetes,A obesidade é um fator de risco importante par...,"[-0.04689414, 0.0009958204, 0.0037504912, 0.01..."
1,Prevenção de Cardiopatias,Fumar danifica os vasos sanguíneos e aumenta o...,"[-0.018584682, -0.039996143, -0.034065615, 0.0..."
2,Como prevenir Alzheimer,"Uma dieta mediterrânea, rica em frutas, verdur...","[-0.026449826, -0.035143714, -0.02825973, 0.03..."


In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [ ]:
consulta = "Porque é bom fazer exercício"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A obesidade é um fator de risco importante para a diabetes. Os exercícios ajudam a controlar os níveis de açúcar no sangue e melhoram a sensibilidade à insulina. Limitar alimentos processados, açúcares adicionados e gorduras saturadas. Optar por frutas, vegetais, 


In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature":0,
}

In [ ]:
prompt = f"Reescreva este texto de modo informal, sem adicionar informações que não faça parte do texto: {trecho}"

model = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config=generation_config)
response = model.generate_content(prompt)



In [ ]:
chat = model.start_chat(history=[])

In [ ]:
prompt = input("Pergunta: ")

while prompt != "fim":
    response = chat.send_message(prompt)
    print("Resposta: ", response.text, "\n")
    prompt = input("Pergunta: ")


KeyboardInterrupt: Interrupted by user